In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib

In [ ]:
# Đọc dữ liệu
df = pd.read_csv('/content/healthcare-dataset-stroke-data.csv')


In [ ]:
# dimensions of data
df.shape

(5110, 12)

In [ ]:
#id , giới tính, tuổi, tăng huyết áp,bệnh tim, tình trạng hôn nhân, công việc, Nơi cư trú, lượng đường tb, chỉ số khối cơ thể, đã từng hút thuốc, đột quỵ
df.columns

Index(['id', 'gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'stroke'],
      dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [ ]:
df.describe()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,4909.000000,5110.000000
mean,36517.829354,43.226614,0.097456,0.054012,106.147677,28.893237,0.048728
std,21161.721625,22.612647,0.296607,0.226063,45.283560,7.854067,0.215320
min,67.000000,0.080000,0.000000,0.000000,55.120000,10.300000,0.000000
25%,17741.250000,25.000000,0.000000,0.000000,77.245000,23.500000,0.000000
50%,36932.000000,45.000000,0.000000,0.000000,91.885000,28.100000,0.000000
75%,54682.000000,61.000000,0.000000,0.000000,114.090000,33.100000,0.000000
max,72940.000000,82.000000,1.000000,1.000000,271.740000,97.600000,1.000000


In [ ]:
# Xử lý dữ liệu thiếu
df['bmi'].fillna(df['bmi'].mean(), inplace=True)

In [ ]:
df['bmi'].isnull().sum()

0

In [ ]:
# tiền xử lí dữ liệu
lb_make = LabelEncoder()

In [ ]:
df['gender'] = lb_make.fit_transform(df['gender'])
df['age'] = lb_make.fit_transform(df['age'])
df['hypertension'] = lb_make.fit_transform(df['hypertension'])
df['heart_disease'] = lb_make.fit_transform(df['heart_disease'])
df['ever_married'] = lb_make.fit_transform(df['ever_married'])
df['work_type'] = lb_make.fit_transform(df['work_type'])
df['Residence_type'] = lb_make.fit_transform(df['Residence_type'])
df['avg_glucose_level'] = lb_make.fit_transform(df['avg_glucose_level'])
df['bmi'] = lb_make.fit_transform(df['bmi'])
df['smoking_status'] = lb_make.fit_transform(df['smoking_status'])
df['stroke'] = lb_make.fit_transform(df['stroke'])
df.head(10)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,1,88,0,1,1,2,1,3850,240,1,1
1,51676,0,82,0,0,1,3,0,3588,162,2,1
2,31112,1,101,0,1,1,2,0,2483,199,2,1
3,60182,0,70,0,0,1,2,1,3385,218,3,1
4,1665,0,100,1,0,1,3,0,3394,113,2,1
5,56669,1,102,0,0,1,2,1,3448,164,1,1
6,53882,1,95,1,1,1,2,0,588,147,2,1
7,10434,0,90,0,0,0,2,1,1951,101,2,1
8,27419,0,80,0,0,1,2,0,912,162,0,1
9,60491,0,99,0,0,1,2,1,126,115,0,1


In [ ]:
# chuẩn hóa dữ liệu của mình về 0 và 1 -> model chạy nhanh hơn và chính xác
scaler = MinMaxScaler(feature_range=(0,1))
scaler_cols=['age','avg_glucose_level','bmi']
df[scaler_cols] = scaler.fit_transform(df[scaler_cols])

In [ ]:
# Xây dựng mô hình
features = ['gender',
            'age',
            'hypertension',
            'heart_disease',
            'ever_married',
            'work_type',
            'Residence_type',
            'avg_glucose_level',
            'bmi',
            'smoking_status']
target =['stroke']
x= df[features]
y= df[target]
print(x)
print(y)

      gender       age  hypertension  heart_disease  ever_married  work_type  \
0          1  0.854369             0              1             1          2   
1          0  0.796117             0              0             1          3   
2          1  0.980583             0              1             1          2   
3          0  0.679612             0              0             1          2   
4          0  0.970874             1              0             1          3   
...      ...       ...           ...            ...           ...        ...   
5105       0  0.980583             1              0             1          2   
5106       0  0.990291             0              0             1          3   
5107       0  0.543689             0              0             1          3   
5108       1  0.699029             0              0             1          2   
5109       0  0.631068             0              0             1          0   

      Residence_type  avg_glucose_level

In [ ]:
# Sử dụng KFold để chia dữ liệu thành các fold
kf = KFold(n_splits=5, shuffle=True, random_state=0)
fold_count = 1

In [ ]:
# Lists để lưu trữ kết quả
confusion_matrices = []
classification_reports = []
accuracies = []

In [ ]:
# Loop qua từng fold và huấn luyện mô hình
for train_index, test_index in kf.split(x):
    X_train, X_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Khởi tạo và huấn luyện mô hình
    model = MultinomialNB()
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)

    # Tính độ chính xác và lưu trữ kết quả
    accuracy = accuracy_score(y_test, prediction) * 100  # Nhân với 100 để chuyển thành phần trăm
    accuracies.append(accuracy)
    confusion_matrices.append(confusion_matrix(y_test, prediction))
    classification_reports.append(classification_report(y_test, prediction))

    # In kết quả của fold hiện tại
    print(f"Fold {fold_count}:")
    print("Confusion Matrix:")
    print(confusion_matrices[-1])
    print("Classification Report:")
    print(classification_reports[-1])
    print(f"Accuracy: {accuracy:.2f}%")  # In độ chính xác dưới dạng phần trăm, làm tròn đến 2 chữ số thập phân
    print("\n")

    fold_count += 1

Fold 1:
Confusion Matrix:
[[968   0]
 [ 54   0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       968
           1       0.00      0.00      0.00        54

    accuracy                           0.95      1022
   macro avg       0.47      0.50      0.49      1022
weighted avg       0.90      0.95      0.92      1022

Accuracy: 94.72%


Fold 2:
Confusion Matrix:
[[979   0]
 [ 43   0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       979
           1       0.00      0.00      0.00        43

    accuracy                           0.96      1022
   macro avg       0.48      0.50      0.49      1022
weighted avg       0.92      0.96      0.94      1022

Accuracy: 95.79%


Fold 3:
Confusion Matrix:
[[973   0]
 [ 49   0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.95      1.00    

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precisio

In [ ]:
# In độ chính xác trung bình của các fold
print("Mean Accuracy:", sum(accuracies) / len(accuracies))

Mean Accuracy: 95.12720156555773


In [ ]:
#  sử dụng mô hình dự đoán với dữ liệu sau
# giới tính nam -1, tuổi tác - 80, cao huyết áp không - 0, bệnh tim không bị - 0
# tình trạng hôn nhân đã kết hôn  - 1, loại công việc tự kinh doanh - 4, Nơi cư trú nông thôn - 0
# lượng đường trung bình trong máu 2500, chỉ số bmi 99, trạng thái đã từng hút thuốc - 3
# nhập bệnh nhân mới
new_patient =[[1,80,0,0,1,4,0,2500,99,3]]
predicted_lable = model.predict(new_patient)# dự đoán
if predicted_lable ==0:
  print("bạn không bị đột quỵ")
else:
  print("bạn đã bị bệnh đột quỵ")

bạn đã bị bệnh đột quỵ


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(


In [ ]:
joblib.dump(model,"Stroke.mdl")

['Stroke.mdl']

In [ ]:
# đọc mô hình từ tệp
model_loaded = joblib.load("Stroke.mdl")
new_patient =[[1,80,0,0,1,4,0,2500,99,3]]
predicted_lable = model.predict(new_patient)# dự đoán
if predicted_lable ==0:
  print("bạn không bị đột quỵ")
else:
  print("bạn đã bị bệnh đột quỵ")

bạn đã bị bệnh đột quỵ


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(
